In [ ]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from datetime import timedelta
from datetime import datetime
import requests
import zipfile 
import os
from dotenv import load_dotenv

Extrair ITR (Formulário de Informações Trimestrais) da CVM e salvar arquivos (1 para cada ano)

In [ ]:
# Carregar variáveis de ambiente
load_dotenv()
path_dados_trimestrais = os.getenv("PATH_DADOS_TRIMESTRAIS")
url_base = os.getenv("URL_BASE_ITR")

# Definindo o período a ser consultado
anos = range(2011, datetime.now().year)

#Escolhendo diretório para salvar os arquivos
os.chdir(f"{path_dados_trimestrais}")

#Loop para realizar requisições e baixar os arquivos da CVM
for ano in anos:
    print(f"Baixando o ano {ano}...")
    download = requests.get(url_base + f'itr_cia_aberta_{ano}.zip')
    open(f"itr_cia_aberta_{ano}.zip", "wb").write(download.content)
    print(f"Finalizado download do ano {ano}...")

print("Fim")

Junta os arquivos trimestrais de dados em um arquivo

In [ ]:
lista_demonstracoes = []
diretorio_atual = os.getcwd()

for arquivo in os.listdir(f"{path_dados_trimestrais}"):
    if arquivo.endswith(".zip"):
        with zipfile.ZipFile(arquivo, "r") as zipped:
            zipped.extractall()
            print(f"Arquivo {arquivo} extraído")
            for arquivo_csv in zipped.namelist():
                if arquivo_csv.endswith(".csv") and "DRE" in arquivo_csv and "con" in arquivo_csv:
                    #pegar apenas os arquivos que tiverem DRE no nome
                    demonstracao = pd.read_csv(arquivo_csv, sep=";", decimal=",", encoding="ISO-8859-1", dtype= {"ORDEM_EXERC": "category"})
                    lista_demonstracoes.append(demonstracao)
                    
                    print(f"*Arquivo {arquivo_csv} adicionado à lista de demonstrações")
                os.remove(arquivo_csv)



Arquivo itr_cia_aberta_2011.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2011.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2012.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2012.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2013.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2013.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2014.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2014.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2015.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2015.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2016.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2016.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2017.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2017.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2018.zip extraído
*Arquivo itr_cia_aberta_DRE_con_2018.csv adicionado à lista de demonstrações
Arquivo itr_cia_aberta_2019.zip extraído
*Arquivo itr_ci

Elimina dados desnecessários e salva arquivo em .csv

In [ ]:
base_dados_tri = pd.concat(lista_demonstracoes)

base_dados_tri.to_csv(path_dados_trimestrais + 'base_dados_tri.csv', sep=";")
print("Arquivo salvo com sucesso!")

Arquivo salvo com sucesso!


Extrai dados DFP (anuais)

In [ ]:
# Carregar variáveis de ambiente
load_dotenv()

path_dados_anuais = os.getenv("PATH_DADOS_ANUAIS")

#definir caminho para salvar os arquivos -- editar aqui caso queira mudar
os.chdir(path_dados_anuais)

#pega dados desde 2010 --- editar aqui para pegar mais anos
anos = range(2010, datetime.now().year)
url_base = os.getenv("URL_BASE_DFP")

for ano in anos:
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    #salvar o arquivo zip no caminho C:\workspace_mestrado\dados_cvm_anuais
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)
    print(f"Arquivo {ano} baixado e salvo")
    

Juntar arquivos separados em um único

In [ ]:
lista_demonstracoes= []
#definir caminho para salvar os arquivos -- editar aqui caso queira mudar

for arquivo in os.listdir(f"{path_dados_anuais}"):
    if arquivo.endswith(".zip"):
        with zipfile.ZipFile(arquivo, "r") as zipped:
            zipped.extractall()
            print(f"Arquivo {arquivo} extraído")
            for arquivo_csv in zipped.namelist():
                if arquivo_csv.endswith(".csv") and "DRE" in arquivo_csv and "con" in arquivo_csv:
                    #pegar apenas os arquivos que tiverem DRE no nome
                    demonstracao = pd.read_csv(arquivo_csv, sep=";", decimal=",", encoding="ISO-8859-1", dtype= {"ORDEM_EXERC": "category"})
                    lista_demonstracoes.append(demonstracao)
                    
                    print(f"*Arquivo {arquivo_csv} adicionado à lista de demonstrações")
                os.remove(arquivo_csv)        


In [ ]:
#agrupa os itens da lista em um dataframe
base_dados = pd.concat(lista_demonstracoes)

#transformar em csv
base_dados.to_csv(path_dados_anuais + 'base_dados_anual.csv', sep=";")
print("Arquivo 'base_dados_2025.csv' salvo com sucesso!")

In [ ]:
#listar empresas no arquivo
lista_cias = base_dados[['DENOM_CIA', 'CD_CVM']]
lista_cias = lista_cias.drop_duplicates()

#transformar salvar arquivo em csv
path_dados_gerais = os.getenv("PATH_DADOS_GERAIS")
lista_cias = pd.DataFrame(lista_cias)
lista_cias.to_csv(path_dados_gerais + 'lista_cias.csv', sep=";")
print("Arquivo 'lista_cias.csv' salvo com sucesso!")